In [1]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

In [19]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

def compare_sheets(file1, file2, result_file):
    # Load the workbooks into pandas dataframes
    df1 = pd.read_excel(file1)
    df2 = pd.read_excel(file2)

    # Add row numbers to each dataframe
    df1['Row_Number_file1'] = df1.index + 1
    df2['Row_Number_file2'] = df2.index + 1

    # Perform a full outer join on the 'Product' column
    merged_df = pd.merge(df1, df2, on='Product', how='outer', suffixes=('_file1', '_file2'))

    # Create a new workbook for the result
    result_wb = openpyxl.Workbook()
    result_ws = result_wb.active
    result_ws.title = "Comparison Result"

    # Add a new sheet for the change log
    change_log_ws = result_wb.create_sheet(title="Change Log")

    # Define fill styles for differences and missing rows
    diff_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    missing_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    # Write the headers for the comparison result sheet
    result_ws.append(["Product", "Color", "Row_Number"])

    # Write the headers for the change log sheet
    change_log_ws.append(["Product", "Old Value", "New Value"])

    # Compare rows and handle missing rows
    for index, row in merged_df.iterrows():
        product = row['Product']
        color_file1 = row['Color_file1'] if pd.notna(row['Color_file1']) else ''
        color_file2 = row['Color_file2'] if pd.notna(row['Color_file2']) else ''
        row_num_file1 = int(row['Row_Number_file1']) if pd.notna(row['Row_Number_file1']) else 'MISSING'
        row_num_file2 = int(row['Row_Number_file2']) if pd.notna(row['Row_Number_file2']) else 'MISSING'

        # Create the comparison results
        if color_file1 != color_file2:
            color_result = f"{color_file1} | {color_file2}"
            change_log_ws.append([product, color_file1, color_file2])
        else:
            color_result = color_file1

        if row_num_file1 != row_num_file2:
            row_num_result = f"{row_num_file1} | {row_num_file2}"
            change_log_ws.append([product, row_num_file1, row_num_file2])
        else:
            row_num_result = row_num_file1

        # Append the result to the worksheet
        result_ws.append([product, color_result, row_num_result])

        # Highlight differences and missing rows
        result_color_cell = result_ws.cell(row=index + 2, column=2)
        if color_file1 != color_file2:
            result_color_cell.fill = diff_fill

        result_row_num_cell = result_ws.cell(row=index + 2, column=3)
        if row_num_file1 == 'MISSING' or row_num_file2 == 'MISSING':
            result_row_num_cell.fill = missing_fill
        elif row_num_file1 != row_num_file2:
            result_row_num_cell.fill = diff_fill

    # Save the result workbook
    result_wb.save(result_file)
    print(f"Comparison complete. Results saved to {result_file}")



In [20]:
# Define file paths
file1 = "./tests/file1.xlsx"
file2 = "./tests/file2.xlsx"
key = "Product"
result_file = "./tests/output5.xlsx"

# Run the comparison
compare_sheets(file1, file2, result_file)


Comparison complete. Results saved to ./tests/output5.xlsx
